## automatic differentiation

In [233]:
import numpy as np
import torch

from torch.utils.tensorboard import SummaryWriter

import torch
import torch.optim as optim
import torch.nn as nn


# check if CUDA is available and if so use it
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/loss_visualization')

np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

theta_list = []
checkpoint_list = []

year = [2017]
period = ['may-jun', 'jun-jul'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')
        N = int(X['N'])
        
    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')
        N = int(X['N'])
        
    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        N = int(X['N']) 
           
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')
        N = int(X['N'])
    

    dist = torch.tensor(X['distance'], dtype=torch.float32)
    tI1 = torch.tensor(X['tI1'].reshape(N,1), dtype=torch.float32)
    tI2 = torch.tensor(X['tI2'].reshape(N,1), dtype=torch.float32)
    sI2 = torch.tensor(X['sI2'].reshape(N,1), dtype=torch.float32)
    
    y_apr = torch.tensor(X['y_apr'].reshape(N,1), dtype=torch.float32)
    y_may = torch.tensor(X['y_may'].reshape(N,1), dtype=torch.float32)
    y_jun = torch.tensor(X['y_jun'].reshape(N,1), dtype=torch.float32)
    y_jul = torch.tensor(X['y_jul'].reshape(N,1), dtype=torch.float32)

    n_apr = torch.tensor(X['n_apr'].reshape(N,1), dtype=torch.float32)
    n_may = torch.tensor(X['n_may'].reshape(N,1), dtype=torch.float32)
    n_jun = torch.tensor(X['n_jun'].reshape(N,1), dtype=torch.float32)
    n_jul = torch.tensor(X['n_jul'].reshape(N,1), dtype=torch.float32)

    a_apr = torch.tensor(X['a_apr'].reshape(N,1), dtype=torch.float32)
    a_may = torch.tensor(X['a_may'].reshape(N,1), dtype=torch.float32)
    a_jun = torch.tensor(X['a_jun'].reshape(N,1), dtype=torch.float32)
    a_jul = torch.tensor(X['a_jul'].reshape(N,1), dtype=torch.float32)

    w_apr = torch.tensor(X['wind_apr'], dtype=torch.float32)
    w_may = torch.tensor(X['wind_may'], dtype=torch.float32)
    w_jun = torch.tensor(X['wind_jun'], dtype=torch.float32)
    w_jul = torch.tensor(X['wind_jul'], dtype=torch.float32)

    sI1_apr = torch.tensor(X['sI1_apr'].reshape(N,1), dtype=torch.float32)
    sI1_may = torch.tensor(X['sI1_may'].reshape(N,1), dtype=torch.float32)
    sI1_jun = torch.tensor(X['sI1_jun'].reshape(N,1), dtype=torch.float32)
    sI1_jul = torch.tensor(X['sI1_jul'].reshape(N,1), dtype=torch.float32)

    s_apr = torch.tensor(X['s_apr'].reshape(N,1), dtype=torch.float32)
    s_may = torch.tensor(X['s_may'].reshape(N,1), dtype=torch.float32)
    s_jun = torch.tensor(X['s_jun'].reshape(N,1), dtype=torch.float32)
    s_jul = torch.tensor(X['s_jul'].reshape(N,1), dtype=torch.float32)

    # Function to normalize the data
    def norm(x):
        
        return (x - torch.min(x)) / (torch.max(x) - torch.min(x))
    
    # Normalize the data
    dist = norm(dist)
    
    a_apr = norm(a_apr)
    a_may = norm(a_may)
    a_jun = norm(a_jun)
    a_jul = norm(a_jul)

    
    # move your tensors to the chosen device
    dist = dist.to(device)
    tI1 = tI1.to(device)
    tI2 = tI2.to(device)
    sI2 = sI2.to(device)
    
    y_apr = y_apr.to(device)
    y_may = y_may.to(device)
    y_jun = y_jun.to(device)
    y_jul = y_jul.to(device)
    
    n_apr = n_apr.to(device)
    n_may = n_may.to(device)
    n_jun = n_jun.to(device)
    n_jul = n_jul.to(device)
    
    a_apr = a_apr.to(device)
    a_may = a_may.to(device)
    a_jun = a_jun.to(device)
    a_jul = a_jul.to(device)
    
    w_apr = w_apr.to(device)
    w_may = w_may.to(device)
    w_jun = w_jun.to(device)
    w_jul = w_jul.to(device)
    
    sI1_apr = sI1_apr.to(device)
    sI1_may = sI1_may.to(device)
    sI1_jun = sI1_jun.to(device)
    sI1_jul = sI1_jul.to(device)
    
    s_apr = s_apr.to(device)
    s_may = s_may.to(device)
    s_jun = s_jun.to(device)
    s_jul = s_jul.to(device)
    
    
    
    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun


        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            eta = []
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
                auto_infection1 = beta1 + delta1 * (y_lag[i] / n_lag[i]) * torch.exp(-eta11 * s_lag[i])
                auto_infection2 = beta2 + delta2 * (y_lag[i] / n_lag[i]) * torch.exp(-eta12 * s_lag[i])

                mask = torch.arange(N) != i # mask out the current yard i

                dispersal_component1 = gamma1 * torch.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * torch.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1).reshape(N-1, 1) * torch.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * torch.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * torch.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1).reshape(N-1, 1) * torch.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask]))

                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)

            eta = torch.stack(eta).view(N,1)

            return eta

        def costFunction(theta):
            neg_log_likelihood = -(1/N) * torch.sum(y * eta(theta) - n * torch.log(1 + torch.exp(eta(theta))))
            return neg_log_likelihood

        # Define number of parameters
        n_params = 12

        # Initialize theta
        theta = torch.empty(n_params, 1, requires_grad=True, device=device)
        nn.init.normal_(theta, mean=0, std=1/torch.sqrt(torch.tensor(n_params).float()))


        # Define the optimizer
        optimizer = optim.Adam([theta], lr=0.01)

        # Training loop
        for t in range(500000):
            optimizer.zero_grad()
            loss = costFunction(theta)
            loss.backward()

            # Compute maximum absolute gradient
            max_grad = torch.max(torch.abs(theta.grad))

            # Check if maximum gradient is less than 1e-5
            if max_grad < 1e-5:
                print("Gradient threshold reached. Ending optimization.")
                break

            optimizer.step()

            if t % 100 == 0: # print every 100 iterations
                print("Period: ", period, "Iteration: ", t, "Loss: ", loss.item(), "Max gradient: ", max_grad.item())

                
        theta_numpy = theta.detach().cpu().numpy()

        if (year == 2017) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2017_auto_4.npy', theta_numpy)
            
        elif (year == 2017) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2017_auto_4.npy', theta_numpy)

Period:  may-jun Iteration:  0 Loss:  146.64553833007812 Max gradient:  80.63551330566406
Period:  may-jun Iteration:  100 Loss:  76.06964111328125 Max gradient:  47.8526725769043
Period:  may-jun Iteration:  200 Loss:  14.352466583251953 Max gradient:  3.12583589553833
Period:  may-jun Iteration:  300 Loss:  13.6926908493042 Max gradient:  1.3027093410491943
Period:  may-jun Iteration:  400 Loss:  13.314697265625 Max gradient:  1.0132067203521729
Period:  may-jun Iteration:  500 Loss:  12.965134620666504 Max gradient:  0.7373781204223633
Period:  may-jun Iteration:  600 Loss:  12.669968605041504 Max gradient:  0.5609830617904663
Period:  may-jun Iteration:  700 Loss:  12.443151473999023 Max gradient:  0.4634530544281006
Period:  may-jun Iteration:  800 Loss:  12.284771919250488 Max gradient:  0.3638085722923279
Period:  may-jun Iteration:  900 Loss:  12.17318344116211 Max gradient:  0.271389365196228
Period:  may-jun Iteration:  1000 Loss:  12.01718521118164 Max gradient:  0.233085751

KeyboardInterrupt: 

In [228]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of available GPU(s): ", torch.cuda.device_count())
    
    # Get the name of the current default GPU
    current_device = torch.cuda.current_device()
    print("Current CUDA Device index: ", current_device)
    print("Current CUDA Device: ", torch.cuda.get_device_name(current_device))
else:
    print("CUDA is not available.")

CUDA is available!
Number of available GPU(s):  1
Current CUDA Device index:  0
Current CUDA Device:  NVIDIA GeForce RTX 3070


## exact optimization

In [23]:
import numpy as np


# Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
def eta(theta):
    eta = []
    
    beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
    
    for i in range(0, N):
        auto_infection1 = beta1 + delta1 * (y_lag[i] / n_lag[i]) * np.exp(-eta11 * s_lag[i])
        auto_infection2 = beta2 + delta2 * (y_lag[i] / n_lag[i]) * np.exp(-eta12 * s_lag[i])

        mask = np.arange(N) != i # mask out the current yard i

        dispersal_component1 = gamma1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1).reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask]))
        dispersal_component2 = gamma2 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1).reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask]))

        eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
        eta.append(eta_i)

    eta = np.array(eta).reshape(N,1)

    return eta


def costFunction(theta):
    neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))
    return neg_log_likelihood


def gradient(theta):
    # Define the gradient of the cost function using numerical differentiation
    epsilon = 1e-8
    grad = np.zeros_like(theta)
    
    for i in range(len(theta)):
        theta_plus = np.array(theta, dtype=float)
        theta_minus = np.array(theta, dtype=float)
        
        theta_plus[i] += epsilon
        theta_minus[i] -= epsilon
        
        grad[i] = (costFunction(theta_plus) - costFunction(theta_minus)) / (2 * epsilon)
    
    return grad


def adam_optimizer(costFunction, gradient, theta_init, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, max_iter=1000):
    theta = np.array(theta_init)
    m = np.zeros_like(theta)
    v = np.zeros_like(theta)
    t = 0
    
    for i in range(max_iter):
        t += 1
        g = gradient(theta)
        m = beta1 * m + (1 - beta1) * g
        v = beta2 * v + (1 - beta2) * (g ** 2)
        
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        
        theta = theta - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    return theta

# Initialize theta and other necessary variables (N, y_lag, n_lag, s_lag, etc.)
theta_init = np.zeros(12)  # replace with your actual initial values

# Run the optimizer to minimize the cost function
#optimal_theta = adam_optimizer(costFunction, gradient, theta_init)

#print("Optimal theta:", optimal_theta)



# old eta

In [ ]:
# Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
def eta(theta):
        
    beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
    
    auto_infection1 = beta1 + delta1 * (y_lag / n_y_lag) * np.exp(-eta11 * s_i_lag)
    auto_infection2 = beta2 + delta2 * (y_lag / n_y_lag) * np.exp(-eta12 * s_i_lag)
    
    dispersal_component1 = gamma1 * np.sum(((area_lag * (z_lag / n_z_lag)) * np.exp(-eta21 * s_j_lag) * (wind_lag * np.exp(-alpha1 * distance_lag)).T * sI1_lag), axis=0, keepdims=True).T
    dispersal_component2 = gamma2 * np.sum(((area_lag * (z_lag / n_z_lag)) * np.exp(-eta22 * s_j_lag) * (wind_lag * np.exp(-alpha2 * distance_lag)).T * sI2_lag), axis=0, keepdims=True).T
    
    eta = tI1_i * (auto_infection1 + dispersal_component1) + tI2_i * (auto_infection2 + dispersal_component2)
    
    return eta

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

theta_list = []
checkpoint_list = []



year = [2014]
period = ['may-jun'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')
        N = X['N']
        
    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')
        N = X['N']
        
    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        N = X['N']
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')
        N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)


    # Function to normalize the data
    def norm(x):
        
        return (x - np.min(x)) / (np.max(x) - np.min(x))
    
    # Normalize the data
    dist = norm(dist)
    
    a_apr = norm(a_apr)
    a_may = norm(a_may)
    a_jun = norm(a_jun)
    a_jul = norm(a_jul)
    
    
    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        

        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            
            eta = []
                
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
            
                auto_infection1 = beta1 + delta1 * (y_lag[i] / n_lag[i]) * np.exp(-eta11 * s_lag[i])
                auto_infection2 = beta2 + delta2 * (y_lag[i] / n_lag[i]) * np.exp(-eta12 * s_lag[i])

                mask = np.arange(N) != i # mask out the current yard i
                
                dispersal_component1 = gamma1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask]))
            
                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)
            
            eta = np.array(eta).reshape(N,1)
            
            return eta


        def costFunction(theta): 
            
            neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))

            return neg_log_likelihood


        def partial(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            d_beta1 = tI1
            d_beta2 = tI2
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                
                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
            
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
            
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)



            grad_entries = np.array([d_beta1, d_beta2, d_delta1, d_delta2, d_gamma1, d_gamma2, d_alpha1, d_alpha2, d_eta11, d_eta12, d_eta21, d_eta22])
            
            return grad_entries

        def partial_by_partial(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            d_beta1 = tI1
            d_beta2 = tI2
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))

                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
                

            grad_entries = np.array([[d_beta1*d_beta1, d_beta2*d_beta1, d_delta1*d_beta1, d_delta2*d_beta1, d_gamma1*d_beta1, d_gamma2*d_beta1, d_alpha1*d_beta1, d_alpha2*d_beta1, d_eta11*d_beta1, d_eta12*d_beta1, d_eta21*d_beta1, d_eta22*d_beta1],
                                    [d_beta1*d_beta2, d_beta2*d_beta2, d_delta1*d_beta2, d_delta2*d_beta2, d_gamma1*d_beta2, d_gamma2*d_beta2, d_alpha1*d_beta2, d_alpha2*d_beta2, d_eta11*d_beta2, d_eta12*d_beta2, d_eta21*d_beta2, d_eta22*d_beta2],
                                    [d_beta1*d_delta1, d_beta2*d_delta1, d_delta1*d_delta1, d_delta2*d_delta1, d_gamma1*d_delta1, d_gamma2*d_delta1, d_alpha1*d_delta1, d_alpha2*d_delta1, d_eta11*d_delta1, d_eta12*d_delta1, d_eta21*d_delta1, d_eta22*d_delta1],
                                    [d_beta1*d_delta2, d_beta2*d_delta2, d_delta1*d_delta2, d_delta2*d_delta2, d_gamma1*d_delta2, d_gamma2*d_delta2, d_alpha1*d_delta2, d_alpha2*d_delta2, d_eta11*d_delta2, d_eta12*d_delta2, d_eta21*d_delta2, d_eta22*d_delta2],
                                    [d_beta1*d_gamma1, d_beta2*d_gamma1, d_delta1*d_gamma1, d_delta2*d_gamma1, d_gamma1*d_gamma1, d_gamma2*d_gamma1, d_alpha1*d_gamma1, d_alpha2*d_gamma1, d_eta11*d_gamma1, d_eta12*d_gamma1, d_eta21*d_gamma1, d_eta22*d_gamma1],
                                    [d_beta1*d_gamma2, d_beta2*d_gamma2, d_delta1*d_gamma2, d_delta2*d_gamma2, d_gamma1*d_gamma2, d_gamma2*d_gamma2, d_alpha1*d_gamma2, d_alpha2*d_gamma2, d_eta11*d_gamma2, d_eta12*d_gamma2, d_eta21*d_gamma2, d_eta22*d_gamma2],
                                    [d_beta1*d_alpha1, d_beta2*d_alpha1, d_delta1*d_alpha1, d_delta2*d_alpha1, d_gamma1*d_alpha1, d_gamma2*d_alpha1, d_alpha1*d_alpha1, d_alpha2*d_alpha1, d_eta11*d_alpha1, d_eta12*d_alpha1, d_eta21*d_alpha1, d_eta22*d_alpha1],
                                    [d_beta1*d_alpha2, d_beta2*d_alpha2, d_delta1*d_alpha2, d_delta2*d_alpha2, d_gamma1*d_alpha2, d_gamma2*d_alpha2, d_alpha1*d_alpha2, d_alpha2*d_alpha2, d_eta11*d_alpha2, d_eta12*d_alpha2, d_eta21*d_alpha2, d_eta22*d_alpha2],
                                    [d_beta1*d_eta11, d_beta2*d_eta11, d_delta1*d_eta11, d_delta2*d_eta11, d_gamma1*d_eta11, d_gamma2*d_eta11, d_alpha1*d_eta11, d_alpha2*d_eta11, d_eta11*d_eta11, d_eta12*d_eta11, d_eta21*d_eta11, d_eta22*d_eta11],
                                    [d_beta1*d_eta12, d_beta2*d_eta12, d_delta1*d_eta12, d_delta2*d_eta12, d_gamma1*d_eta12, d_gamma2*d_eta12, d_alpha1*d_eta12, d_alpha2*d_eta12, d_eta11*d_eta12, d_eta12*d_eta12, d_eta21*d_eta12, d_eta22*d_eta12],
                                    [d_beta1*d_eta21, d_beta2*d_eta21, d_delta1*d_eta21, d_delta2*d_eta21, d_gamma1*d_eta21, d_gamma2*d_eta21, d_alpha1*d_eta21, d_alpha2*d_eta21, d_eta11*d_eta21, d_eta12*d_eta21, d_eta21*d_eta21, d_eta22*d_eta21],
                                    [d_beta1*d_eta22, d_beta2*d_eta22, d_delta1*d_eta22, d_delta2*d_eta22, d_gamma1*d_eta22, d_gamma2*d_eta22, d_alpha1*d_eta22, d_alpha2*d_eta22, d_eta11*d_eta22, d_eta12*d_eta22, d_eta21*d_eta22, d_eta22*d_eta22]])
            
            
            
            return grad_entries

        def partial_sq(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            # delta1 second derivatives
            
            d_delta1_d_eta11 = -tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag) * s_lag
            d_delta1_d_eta12 = 0
            d_delta2_d_eta11 = 0
            d_delta2_d_eta12 = -tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag) * s_lag
            d_gamma1_d_eta22 = 0
            d_gamma1_d_alpha2 = 0
            d_gamma2_d_eta21 = 0
            d_gamma2_d_alpha1 = 0
            d_alpha1_d_gamma2 = 0
            d_alpha1_d_eta22 = 0
            d_alpha1_d_alpha2 = 0
            d_alpha2_d_gamma1 = 0
            d_alpha2_d_eta21 = 0
            d_alpha2_d_alpha1 = 0
            d_eta11_d_delta1 = -tI1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_delta2 = 0
            d_eta11_d_eta11 = tI1 * delta1 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_eta12 = 0
            d_eta12_d_delta1 = 0
            d_eta12_d_delta2 = -tI2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta12_d_eta11 = 0
            d_eta12_d_eta12 = tI2 * delta2 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta21_d_gamma2 = 0
            d_eta21_d_eta22 = 0
            d_eta21_d_alpha2 = 0
            d_eta22_d_gamma1 = 0
            d_eta22_d_eta21 = 0
            d_eta22_d_alpha1 = 0
            
            # summations
            
            d_gamma1_d_eta21 = []
            d_gamma1_d_alpha1 = []
            d_gamma2_d_eta22 = []
            d_gamma2_d_alpha2 = []
            d_alpha1_d_gamma1 = []
            d_alpha1_d_eta21 = []
            d_alpha1_d_alpha1 = []
            d_alpha2_d_gamma2 = []
            d_alpha2_d_eta22 = []
            d_alpha2_d_alpha2 = []
            d_eta21_d_gamma1 = []
            d_eta21_d_eta21 = []
            d_eta21_d_alpha1 = []
            d_eta22_d_gamma2 = []
            d_eta22_d_eta22 = []
            d_eta22_d_alpha2 = []
            
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
                
                d_gamma1_d_eta21_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_gamma1_d_alpha1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_gamma2_d_eta22_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_gamma2_d_alpha2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha1_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_alpha2_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha2_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_eta21_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta21_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (s_lag[mask]**2))
                d_eta21_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))
                d_eta22_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_eta22_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (s_lag[mask]**2))
                d_eta22_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))

                d_gamma1_d_eta21.append(d_gamma1_d_eta21_i)
                d_gamma1_d_alpha1.append(d_gamma1_d_alpha1_i)
                d_gamma2_d_eta22.append(d_gamma2_d_eta22_i)
                d_gamma2_d_alpha2.append(d_gamma2_d_alpha2_i)
                d_alpha1_d_gamma1.append(d_alpha1_d_gamma1_i)
                d_alpha1_d_eta21.append(d_alpha1_d_eta21_i)
                d_alpha1_d_alpha1.append(d_alpha1_d_alpha1_i)
                d_alpha2_d_gamma2.append(d_alpha2_d_gamma2_i)
                d_alpha2_d_eta22.append(d_alpha2_d_eta22_i)
                d_alpha2_d_alpha2.append(d_alpha2_d_alpha2_i)
                d_eta21_d_gamma1.append(d_eta21_d_gamma1_i)
                d_eta21_d_eta21.append(d_eta21_d_eta21_i)
                d_eta21_d_alpha1.append(d_eta21_d_alpha1_i)
                d_eta22_d_gamma2.append(d_eta22_d_gamma2_i)
                d_eta22_d_eta22.append(d_eta22_d_eta22_i)
                d_eta22_d_alpha2.append(d_eta22_d_alpha2_i)
                
            d_gamma1_d_eta21 = np.array(d_gamma1_d_eta21).reshape((N, 1))
            d_gamma1_d_alpha1 = np.array(d_gamma1_d_alpha1).reshape((N, 1))
            d_gamma2_d_eta22 = np.array(d_gamma2_d_eta22).reshape((N, 1))
            d_gamma2_d_alpha2 = np.array(d_gamma2_d_alpha2).reshape((N, 1))
            d_alpha1_d_gamma1 = np.array(d_alpha1_d_gamma1).reshape((N, 1))
            d_alpha1_d_eta21 = np.array(d_alpha1_d_eta21).reshape((N, 1))
            d_alpha1_d_alpha1 = np.array(d_alpha1_d_alpha1).reshape((N, 1))
            d_alpha2_d_gamma2 = np.array(d_alpha2_d_gamma2).reshape((N, 1))
            d_alpha2_d_eta22 = np.array(d_alpha2_d_eta22).reshape((N, 1))
            d_alpha2_d_alpha2 = np.array(d_alpha2_d_alpha2).reshape((N, 1))
            d_eta21_d_gamma1 = np.array(d_eta21_d_gamma1).reshape((N, 1))
            d_eta21_d_eta21 = np.array(d_eta21_d_eta21).reshape((N, 1))
            d_eta21_d_alpha1 = np.array(d_eta21_d_alpha1).reshape((N, 1))
            d_eta22_d_gamma2 = np.array(d_eta22_d_gamma2).reshape((N, 1))
            d_eta22_d_eta22 = np.array(d_eta22_d_eta22).reshape((N, 1))
            d_eta22_d_alpha2 = np.array(d_eta22_d_alpha2).reshape((N, 1))
                
                
            
            zero = np.zeros((N, 1))
            
            hess_entries = np.array([[zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta2
                                    [zero, zero, zero, zero, zero, zero, zero, zero, d_eta11_d_delta1, zero, zero, zero],    #d_delta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, d_eta12_d_delta2, zero, zero],    #d_delta2
                                    [zero, zero, zero, zero, zero, zero, d_alpha1_d_gamma1, zero, zero, zero, d_eta21_d_gamma1, zero],    #d_gamma1
                                    [zero, zero, zero, zero, zero, zero, zero, d_alpha2_d_gamma2, zero, zero, zero, d_eta22_d_gamma2],    #d_gamma2
                                    [zero, zero, zero, zero, d_gamma1_d_alpha1, zero, d_alpha1_d_alpha1, zero, zero, zero, d_eta21_d_alpha1, zero],    #d_alpha1
                                    [zero, zero, zero, zero, zero, d_gamma2_d_alpha2, zero, d_alpha2_d_alpha2, zero, zero, zero, d_eta22_d_alpha2],    #d_alpha2
                                    [zero, zero, d_delta1_d_eta11, zero, zero, zero, zero, zero, d_eta11_d_eta11, zero, zero, zero],    #d_eta11
                                    [zero, zero, zero, d_delta2_d_eta12, zero, zero, zero, zero, zero, d_eta12_d_eta12, zero, zero],    #d_eta12
                                    [zero, zero, zero, zero, d_gamma1_d_eta21, zero, d_alpha1_d_eta21, zero, zero, zero, d_eta21_d_eta21, zero],    #d_eta21
                                    [zero, zero, zero, zero, zero, d_gamma2_d_eta22, zero, d_alpha2_d_eta22, zero, zero, zero, d_eta22_d_eta22]])   #d_eta22
            
            
            return hess_entries



        # Gradient entries
        def gradient(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Gradient 
            gradient = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
            
            return gradient
        
        # Gradient entries
        def gradient1(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Gradient 
            gradient1 = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
            
            return gradient1.ravel()


        # Hessian
        def hessian(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Hessian entries
            hessian = - (1 / N) * np.sum((partial_sq(theta) * mu - n * (partial_by_partial(theta)) * (np.exp(-eta(theta)) / (1 + np.exp(-eta(theta)))**2)), axis=2)
            hessian = hessian.reshape((12, 12))
            
            return hessian
        
        
        # Adam optimizer
        def adam(theta, alpha, num_iters, tolerance, b_1=0.9, b_2=0.999, eps=1e-8):

            # Make a copy of theta, to avoid changing the original array, since numpy arrays are passed by reference to functions
            theta = theta.copy()
            
            # Use a python list to save cost in every iteration
            J_history = []
            
            m = np.zeros(theta.shape)
            v = np.zeros(theta.shape)
            
            for i in range(num_iters):
                
                # Gradient
                g = gradient(theta)

                # First moment
                m = b_1 * m + (1 - b_1) * g

                # Second moment
                v = b_2 * v + (1 - b_2) * g**2

                # Bias correction for the first and second moments

                mhat = m / (1 - b_1**(i+1))

                vhat = v / (1 - b_2**(i+1))
                
                change = alpha * mhat / (np.sqrt(vhat) + eps)
                
                # Update parameter theta
                theta = theta - change
                
                # save the cost J in every iteration
                J_history.append(costFunction(theta))
                # report progress
                
                if i % 100 == 0: # print every 100 iterations
                    #print('Year: ', year, 'Period: ', period, '#', i , 'cost: ', costFunction(theta), 'MaxGrad: ', np.max(np.abs(gradient(theta))), end='\r')
                    print('Year: ', year, 'Period: ', period, '#', i , 'cost: ', costFunction(theta), 'MaxGrad: ', np.max(np.abs(g)), 'MinEig: ', np.min(np.linalg.eigvalsh(hessian(theta))), end='\r')
                
                #if (np.max(np.abs(gradient(theta))) <= tolerance) & (np.min(np.linalg.eigh(hessian(theta), UPLO='L')[0]) > 0):
                #   break
                
                if (np.max(np.abs(g)) <= tolerance):
                    break
                
            return theta, J_history, g 

            
        # Initialize fitting parameters

        # Xavier initialization for theta
        #theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))

        # Use below theta to continue from checkpoint
        #checkpoint = np.load('../reports/apr_may_2014.npy', allow_pickle=True)
        
        if year == 2014:
        
            alpha = 0.02
            tolerance = 1e-4
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        if year == 2015:
            
            alpha = 0.02
            tolerance = 1e-4
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        elif year == 2016:
        
            alpha = 0.02
            tolerance = 1e-4
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        elif year == 2017:
            
            alpha = 0.000001
            tolerance = 1e-12
            
            # Initialize fitting parameters
            #theta = np.random.normal(-0.5, 0.5, size=(12, 1))
            
            # Minimize the cost function and get the optimized initial parameter values
            #res = minimize(costFunction, theta, method='BFGS', jac=gradient1, hess=hessian, options={'gtol': 1e-8, 'maxiter': 100000, 'disp': True})
            #theta = res.x
            #theta = theta.reshape(12, 1)
            
            if period == 'may-jun':
                
                theta = np.load('../reports/parameters/theta_may-jun_2017_num_1.npy', allow_pickle=True)
            
            elif period == 'jun-jul':
                
                theta = np.load('../reports/parameters/theta_jun-jul_2017_num_1.npy', allow_pickle=True)
                
            theta = theta.reshape(12, 1)
            
        #theta = np.load('../reports/theta2.npy')

        # Gradient descent settings
        iterations = 2000000

        theta, J_history, g = adam(theta, alpha, iterations, tolerance)

        #print('iteration start:\t{:.3f}'.format(np.int32(checkpoint[1])))
        #print('previous final cost:\t{:.3f}'.format(checkpoint[2]))
        #print('updated final cost:\t{:.3f}'.format(J_history[-1]))
        #print('theta: \n', theta)

        #plt.plot(list(range(1, len(J_history) + 1)), J_history)
        #plt.xlabel('iterations')
        #plt.ylabel('cost')
        
        
        #plt.show()

        # Save trained parameters

        #iterations += checkpoint[1]
        checkpoint = np.array([theta, iterations, J_history, g], dtype=object)
        
        theta_list.append(theta)
        checkpoint_list.append(checkpoint)

        if (year == 2014) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2014.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2014.npy', checkpoint)
        
        elif (year == 2014) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2014.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2014.npy', checkpoint)
            
        elif (year == 2015) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2015.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2015.npy', checkpoint)
            
        elif (year == 2015) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2015.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2015.npy', checkpoint)
            
        elif (year == 2016) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2016.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2016.npy', checkpoint)
            
        elif (year == 2016) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2016.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2016.npy', checkpoint)
            
        elif (year == 2017) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2017_1_1.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2017_1_3.npy', checkpoint)
            
        elif (year == 2017) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2017_1_2.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2017_1_3.npy', checkpoint)

KeyboardInterrupt: 

In [2]:
theta_may_jun_1 = np.load('../reports/parameters/theta_may-jun_2014_1.npy', allow_pickle=True)
theta_may_jun_SLSQP = np.load('../reports/parameters/theta_may-jun_2014_SLSQP.npy', allow_pickle=True)

In [3]:
theta_may_jun_1

array([[  -1.90199427],
       [  -5.89298669],
       [1084.87231619],
       [  98.22151184],
       [ 826.58571766],
       [4190.68361118],
       [  12.36124891],
       [  19.77812373],
       [   3.90317286],
       [   1.55499324],
       [  -0.226496  ],
       [   0.7442895 ]])

In [4]:
theta_may_jun_SLSQP

array([[  -1.90834995],
       [  -5.8923251 ],
       [1068.04938672],
       [  98.19933073],
       [1070.82044118],
       [4188.20143807],
       [  11.91421093],
       [  19.77882165],
       [   3.88684962],
       [   1.55486263],
       [   0.0001    ],
       [   0.7437819 ]])

In [7]:
gradient(theta_may_jun_1)

array([[-0.00000054],
       [-0.00000025],
       [-0.00000005],
       [-0.        ],
       [ 0.00009916],
       [-0.00000002],
       [-0.00000002],
       [ 0.00000005],
       [-0.00000001],
       [ 0.00000024],
       [ 0.00000349],
       [ 0.00000114]])

In [8]:
gradient(theta_may_jun_SLSQP)

array([[ 0.00000955],
       [ 0.00041353],
       [-0.00000006],
       [ 0.00000069],
       [-0.00000013],
       [ 0.00000041],
       [-0.00000056],
       [-0.00008012],
       [ 0.00001308],
       [-0.00015229],
       [ 0.19595247],
       [-0.00206554]])

In [10]:
np.linalg.eigvalsh(hessian(theta_may_jun_SLSQP))

array([ 0.        ,  0.00000002,  0.00000014,  0.0001289 ,  0.00618432,
        0.00980424,  0.59164684,  1.28884469,  1.38236434,  5.65514495,
       11.97207314, 48.4394634 ])

In [5]:
costFunction(theta_may_jun_1)

22.13438758691359

In [6]:
costFunction(theta_may_jun_SLSQP)

22.16859251889821

In [294]:
gradient(theta)

array([[ 0.],
       [ 0.],
       [-0.],
       [-0.],
       [ 0.],
       [-0.],
       [ 0.],
       [-0.],
       [-0.],
       [-0.],
       [ 0.],
       [-0.]])

In [14]:
np.set_printoptions(suppress=True)
theta_may_jun

array([    2.76562827,     0.38540121, -2026.35615832,    34.42700705,
       -6426.03181702, -9151.57587319,    -4.60772949,    -4.940867  ,
        2184.36919864,    -1.11189886,    -0.4499415 ,     0.17128957])

In [12]:
theta_jun_jul

array([[ -3.27984287],
       [ -1.88645822],
       [ 13.99890895],
       [ 14.97972467],
       [-10.87910002],
       [-39.35831275],
       [ -2.06276433],
       [  2.61942527],
       [ -0.64336604],
       [  0.20618654],
       [ -0.68528102],
       [ -1.46375126]])

In [11]:
np.set_printoptions(suppress=False)
np.linalg.eigvalsh(hessian(theta_jun_jul))

array([1.53115046e-05, 4.48719789e-05, 1.71262828e-03, 4.41033098e-03,
       4.64373057e-03, 1.41855302e-01, 2.64989420e-01, 1.04621614e+00,
       2.36655165e+00, 2.87910505e+01, 1.04866123e+02, 1.07810199e+02])

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

## Derivatives

$$
\begin{align*}
\frac{\partial \eta_{i}}{\partial \beta_{k}} &= I_{k}^{(t)}(i) \\

\frac{\partial \eta_{i}}{\partial \delta_{k}} &= I_{k}^{(t)}(i)\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{1 k}} &= -I_{k}^{(t)}(i) \delta_{k} s_{i}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{2 k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z j}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) s_{j}\right] \\

\frac{\partial \eta_{i}}{\partial \gamma_{k}} &= I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j)\right] \\

\frac{\partial \eta_{i}}{\partial \alpha_{k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) d_{i j}\right]
\end{align*}
$$

### Cost Function

$$J(\theta) = 
-\frac{1}{N} \sum_{i=1}^{N} y_{i} \eta_{i}-n_{i} \log \left(1+e^{\eta_{i}}\right)
$$

### Gradient

$$\frac{\partial J}{\partial \theta} =
-\frac{1}{N}\sum_{i=1}^{N} \frac{\partial \eta_{i}}{\partial \theta}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)
$$


### Hessian

$$\frac{\partial^2 J}{\partial \theta^2} = -\frac{1}{N}
\sum_{i=1}^{N}\left[\frac{\partial^{2} \eta_{i}}{\partial \theta^{2}}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)- n_{i} \left(\frac{\partial \eta_{i}}{\partial \theta}\right)^{2}\frac{e^{-\eta_{i}}}{\left(1+e^{-\eta_{i}}\right)^2}\right]
$$

## Optimization

In [50]:
#checkpoint = np.array([theta, iterations, J_history], dtype=object)
#np.save('../reports/checkpoint_June_July2_backup.npy', checkpoint)

In [45]:
checkpoint = np.load('../reports/checkpoint_June_July2.npy', allow_pickle=True)
theta = checkpoint[0]

In [238]:
theta

array([[    1.5457632 ],
       [   -0.07547458],
       [   -4.8742034 ],
       [   51.98424236],
       [-3122.39869964],
       [-4194.55086314],
       [   -5.29015772],
       [   -5.85815199],
       [   -6.70538574],
       [   -0.91393116],
       [   -0.59329742],
       [    0.02099419]])

## Parameter Estimation

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

$$\beta_1 = -2.89902093$$
$$\beta_2 = -4.33376942$$
$$\delta_1 = 3.86406603$$
$$\delta_2 = 7.1830044$$
$$\gamma_1 = 0.06209235$$
$$\gamma_2 = 6.21221296$$
$$\alpha_1 = 0.17578305$$
$$\alpha_2 = 1.31264131$$
$$\eta_{11} = 0.13978209$$
$$\eta_{12} = 0.40521989$$
$$\eta_{21} = -0.79182359$$
$$\eta_{22} = 0.55742334$$

## Eigenvalues of Hessian

In [61]:
np.linalg.eigvalsh(hessian(theta_numpy))

array([   0.        ,    0.00000037,    0.00000144,    0.00000554,
          0.00212706,    0.00660576,    0.02351497,    0.40513871,
          2.03226417,    3.69983217,  715.52475767, 2615.2486514 ])

## Gradient

In [11]:
gradient(theta)

array([[ 0.00035165],
       [ 0.00000092],
       [ 0.00093506],
       [-0.00000046],
       [ 0.00370447],
       [-0.00027325],
       [-0.00982039],
       [-0.00226836],
       [-0.00229518],
       [ 0.00000655],
       [-0.00047533],
       [ 0.00037915]])

In [31]:
nGrad = nd.Gradient(costFunction)
nHess = nd.Hessian(costFunction)
hess_theta = nHess(theta.reshape(12,))
grad_theta = nGrad(theta.reshape(12,))

In [10]:
def prob(theta):
    
    p = 1 / (1 + np.exp(-eta(theta)))
    
    return p

In [12]:
print('estimated probability of disease: \n', prob(theta))

estimated probability of disease: 
 [[0.02611759]
 [0.02631787]
 [0.02620498]
 [0.03368977]
 [0.02623533]
 [0.02624147]
 [0.02745905]
 [0.02710209]
 [0.04514933]
 [0.04289552]
 [0.16258389]
 [0.03171659]
 [0.15754236]
 [0.03329544]
 [0.02628519]
 [0.02748307]
 [0.02906667]
 [0.02786479]
 [0.02657169]
 [0.02892042]
 [0.02622809]
 [0.04582204]
 [0.5       ]
 [0.84480418]
 [0.24196183]
 [0.2553849 ]
 [0.33620011]
 [0.02684969]
 [0.49306239]
 [0.80226981]
 [0.53574716]
 [0.41690551]
 [0.91858263]
 [0.14165495]
 [0.        ]
 [0.02439342]
 [0.02535986]
 [0.02457473]
 [0.02567178]
 [0.17610494]
 [0.02407955]
 [0.01639678]
 [0.01495841]
 [0.02234735]
 [0.02345555]
 [0.02276539]
 [0.5       ]
 [0.02272612]
 [0.02457618]
 [0.11173597]
 [0.10794024]
 [0.22234288]
 [0.14008519]
 [0.2470191 ]
 [0.24700966]
 [0.02734554]
 [0.25025533]
 [0.02731197]
 [0.2534015 ]
 [0.02732704]
 [0.24922973]
 [0.02735182]
 [0.02728799]
 [0.02733912]
 [0.02321656]
 [0.02306974]
 [0.02242573]
 [0.5       ]
 [0.02569351

In [3]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def compute_spiral(a, b, max_power):
    """Compute the complex number spiral for (a + bi)^n."""
    z = a + 1j * b
    powers = [z ** n for n in range(1, max_power + 1)]
    return [np.real(p) for p in powers], [np.imag(p) for p in powers]

# Initial parameters
a_init = 1.0
b_init = 1.0
max_power_init = 5

x, y = compute_spiral(a_init, b_init, max_power_init)

fig = make_subplots(rows=1, cols=1)

scatter = go.Scatter(x=x, y=y, mode='lines+markers')
fig.add_trace(scatter)

# Sliders
fig.update_layout(
    updatemenu=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Re+",
                          method="relayout",
                          args=["x", [val + 0.1 for val in x]]),
                     dict(label="Re-",
                          method="relayout",
                          args=["x", [val - 0.1 for val in x]]),
                     dict(label="Im+",
                          method="relayout",
                          args=["y", [val + 0.1 for val in y]]),
                     dict(label="Im-",
                          method="relayout",
                          args=["y", [val - 0.1 for val in y]])]
        )
    ]
)

# Show plot
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'updatemenu'

Did you mean "updatemenus"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note tha